In [334]:
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("..")
from utils import *
from linear_regression import *
from svm import *
from softmax import *
from features import *
from kernel import *
from sklearn.svm import LinearSVC
from scipy import sparse

In [344]:
verbose = False

epsilon = 1e-6

def green(s):
    return '\033[1;32m%s\033[m' % s

def yellow(s):
    return '\033[1;33m%s\033[m' % s

def red(s):
    return '\033[1;31m%s\033[m' % s

def log(*m):
    print(" ".join(map(str, m)))

def log_exit(*m):
    log(red("ERROR:"), *m)
    exit(1)


def check_real(ex_name, f, exp_res, *args):
    try:
        res = f(*args)
    except NotImplementedError:
        log(red("FAIL"), ex_name, ": not implemented")
        return True
    if not np.isreal(res):
        log(red("FAIL"), ex_name, ": does not return a real number, type: ", type(res))
        return True
    if not -epsilon < res - exp_res < epsilon:
        log(red("FAIL"), ex_name, ": incorrect answer. Expected", exp_res, ", got: ", res)
        return True


def equals(x, y):
    if type(y) == np.ndarray:
        return (np.abs(x - y) < epsilon).all()
    return -epsilon < x - y < epsilon

def check_array(ex_name, f, exp_res, *args):
    try:
        res = f(*args)
    except NotImplementedError:
        log(red("FAIL"), ex_name, ": not implemented")
        return True
    if not type(res) == np.ndarray:
        log(red("FAIL"), ex_name, ": does not return a numpy array, type: ", type(res))
        return True
    if not len(res) == len(exp_res):
        log(red("FAIL"), ex_name, ": expected an array of shape ", exp_res.shape, " but got array of shape", res.shape)
        return True
    if not equals(res, exp_res):
        log(red("FAIL"), ex_name, ": incorrect answer. Expected", exp_res, ", got: ", res)

        return True

In [335]:
def closed_form(X, Y, lambda_factor):
    """
    Computes the closed form solution of linear regression with L2 regularization

    Args:
        X - (n, d + 1) NumPy array (n datapoints each with d features plus the bias feature in the first dimension)
        Y - (n, ) NumPy array containing the labels (a number from 0-9) for each
            data point
        lambda_factor - the regularization constant (scalar)
    Returns:
        theta - (d + 1, ) NumPy array containing the weights of linear regression. Note that theta[0]
        represents the y-axis intercept of the model and therefore X[0] = 1
    """

    A = (X.T @ X) + (np.identity(X.shape[1]) * lambda_factor)
    b = X.T @ Y

    rtdo = np.linalg.inv(A) @ b
    
    return rtdo

In [ ]:
X = np.array([[0.95892691, 0.29797309, 0.42508341]
,[0.11065406, 0.73685701, 0.62536117]
,[0.64200465, 0.03767413, 0.95782812]
,[0.82838564, 0.58323297, 0.4352333]
,[0.19839889, 0.80945848, 0.8674617]
,[0.98415849, 0.57810422, 0.19369748]
,[0.4715416, 0.96105139, 0.85578663]
,[0.03546793, 0.6986972, 0.75711946]
,[0.30809539, 0.86729695, 0.40138412]
,[0.99220048, 0.43055934, 0.30102598]
,[0.75688374, 0.97314883, 0.17028019]
,[0.80568499, 0.97973042, 0.05511123]
,[0.60451808, 0.10718601, 0.78651937]
,[0.91556592, 0.90700619, 0.12362445]
,[0.64209056, 0.05692244, 0.44575169]
,[0.39288006, 0.47537458, 0.52895656]])

Y = np.array([0.14923122, 0.49691812, 0.05387376, 0.86180237, 0.94807534, 0.25180968
,0.12675531, 0.0356647, 0.33843285, 0.95558534, 0.23558956, 0.30578756
,0.37154414, 0.81286067, 0.90505945, 0.60942559])

lambda_factor = 0.5126229286392731

In [ ]:
closed_form(X, Y, lambda_factor)

In [ ]:
def one_vs_rest_svm(train_x, train_y, test_x):
    """
    Trains a linear SVM for binary classifciation

    Args:
        train_x - (n, d) NumPy array (n datapoints each with d features)
        train_y - (n, ) NumPy array containing the labels (0 or 1) for each training data point
        test_x - (m, d) NumPy array (m datapoints each with d features)
    Returns:
        pred_test_y - (m,) NumPy array containing the labels (0 or 1) for each test data point
    """
    lsvc = LinearSVC(random_state = 0, C=0.1)

    lsvc.fit(train_x, train_y)
    
    pred_test_y = lsvc.predict(test_x)
    
    return pred_test_y

In [ ]:
one_vs_rest_svm()

In [ ]:
def multi_class_svm(train_x, train_y, test_x):
    """
    Trains a linear SVM for multiclass classifciation using a one-vs-rest strategy

    Args:
        train_x - (n, d) NumPy array (n datapoints each with d features)
        train_y - (n, ) NumPy array containing the labels (int) for each training data point
        test_x - (m, d) NumPy array (m datapoints each with d features)
    Returns:
        pred_test_y - (m,) NumPy array containing the labels (int) for each test data point
    """

    lsvc = LinearSVC(random_state = 0, C=0.1, multi_class= 'ovr')

    lsvc.fit(train_x, train_y)
    
    pred_test_y = lsvc.predict(test_x)
    
    return pred_test_y

In [ ]:
multi_class_svm

In [336]:
def compute_probabilities(X, theta, temp_parameter):
    """
    Computes, for each datapoint X[i], the probability that X[i] is labeled as j
    for j = 0, 1, ..., k-1

    Args:
        X - (n, d) NumPy array (n datapoints each with d features)
        theta - (k, d) NumPy array, where row j represents the parameters of our model for label j
        temp_parameter - the temperature parameter of softmax function (scalar)
    Returns:
        H - (k, n) NumPy array, where each entry H[j][i] is the probability that X[i] is labeled as j
    """
    xt = (theta @ X.T) / temp_parameter
    c = np.tile(np.max(xt, axis = 0), (xt.shape[0], 1))

    xtc = xt - c

    H = np.exp(xtc) / np.sum(np.exp(xtc), axis=0) 

    return H


In [337]:
def compute_cost_function(X, Y, theta, lambda_factor, temp_parameter):
    """
    Computes the total cost over every datapoint.

    Args:
        X - (n, d) NumPy array (n datapoints each with d features)
        Y - (n, ) NumPy array containing the labels (a number from 0-9) for each
            data point
        theta - (k, d) NumPy array, where row j represents the parameters of our
                model for label j
        lambda_factor - the regularization constant (scalar)
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns
        c - the cost value (scalar)
    """
    H = compute_probabilities(X, theta, temp_parameter)
    regression_term = np.sum(np.log(np.max(H, axis = 0))) / H.shape[1]
    regularization_term = np.sum(np.power(theta, 2)) * (lambda_factor / 2)
    
    return regularization_term - regression_term

In [ ]:
ex_name = "Compute cost function"
n, d, k = 3, 5, 7
X = np.arange(0, n * d).reshape(n, d)
Y = np.arange(0, n)
zeros = np.zeros((k, d))
temp = 0.2
lambda_factor = 0.5
exp_res = 1.9459101490553135

In [367]:
def run_gradient_descent_iteration(X, Y, theta, alpha, lambda_factor, temp_parameter):
    """
    Runs one step of batch gradient descent

    Args:
        X - (n, d) NumPy array (n datapoints each with d features)
        Y - (n, ) NumPy array containing the labels (a number from 0-9) for each
            data point
        theta - (k, d) NumPy array, where row j represents the parameters of our
                model for label j
        alpha - the learning rate (scalar)
        lambda_factor - the regularization constant (scalar)
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns:
        theta - (k, d) NumPy array that is the final value of parameters theta
    """
    n = X.shape[0]
    k = theta.shape[0]
    d = theta.shape[1]

    M = sparse.coo_matrix(([1]*n, (Y, range(n))), shape=(k,n)).toarray()
    p = compute_probabilities(X, theta, temp_parameter)

    termino_1 = ((M - p)  @ X) / (n * temp_parameter)
    termino_2 = lambda_factor * theta
    gdte = termino_2 - termino_1

    return theta - (alpha * gdte)

In [368]:
n, d, k = 3, 5, 7
X = np.arange(0, n * d).reshape(n, d)
Y = np.arange(0, n)
zeros = np.zeros((k, d))
alpha = 2
temp_parameter = 0.2
lambda_factor = 0.5

run_gradient_descent_iteration(X, Y, zeros, alpha, lambda_factor, temp_parameter)

array([[ -7.14285714,  -5.23809524,  -3.33333333,  -1.42857143,
          0.47619048],
       [  9.52380952,  11.42857143,  13.33333333,  15.23809524,
         17.14285714],
       [ 26.19047619,  28.0952381 ,  30.        ,  31.9047619 ,
         33.80952381],
       [ -7.14285714,  -8.57142857, -10.        , -11.42857143,
        -12.85714286],
       [ -7.14285714,  -8.57142857, -10.        , -11.42857143,
        -12.85714286],
       [ -7.14285714,  -8.57142857, -10.        , -11.42857143,
        -12.85714286],
       [ -7.14285714,  -8.57142857, -10.        , -11.42857143,
        -12.85714286]])

In [364]:
def update_y(train_y, test_y):
    """
    Changes the old digit labels for the training and test set for the new (mod 3)
    labels.

    Args:
        train_y - (n, ) NumPy array containing the labels (a number between 0-9)
                 for each datapoint in the training set
        test_y - (n, ) NumPy array containing the labels (a number between 0-9)
                for each datapoint in the test set

    Returns:
        train_y_mod3 - (n, ) NumPy array containing the new labels (a number between 0-2)
                     for each datapoint in the training set
        test_y_mod3 - (n, ) NumPy array containing the new labels (a number between 0-2)
                    for each datapoint in the test set
    """
    return (train_y % 3, test_y % 3)


array([[  3.5714286,   2.6190476,   1.6666667,   0.7142857,  -0.2380952],
       [ -4.7619048,  -5.7142857,  -6.6666667,  -7.6190476,  -8.5714286],
       [-13.0952381, -14.047619 , -15.       , -15.952381 , -16.9047619],
       [  3.5714286,   4.2857143,   5.       ,   5.7142857,   6.4285714],
       [  3.5714286,   4.2857143,   5.       ,   5.7142857,   6.4285714],
       [  3.5714286,   4.2857143,   5.       ,   5.7142857,   6.4285714],
       [  3.5714286,   4.2857143,   5.       ,   5.7142857,   6.4285714]])

In [369]:
train_y = np.arange(0, 10)
test_y = np.arange(9, -1, -1)
exp_res = (
        np.array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0]),
        np.array([0, 2, 1, 0, 2, 1, 0, 2, 1, 0])
        )

In [373]:
def get_classification(X, theta, temp_parameter):
    """
    Makes predictions by classifying a given dataset

    Args:
        X - (n, d - 1) NumPy array (n data points, each with d - 1 features)
        theta - (k, d) NumPy array where row j represents the parameters of our model for
                label j
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns:
        Y - (n, ) NumPy array, containing the predicted label (a number between 0-9) for
            each data point
    """
    X = augment_feature_vector(X)
    probabilities = compute_probabilities(X, theta, temp_parameter)
    return np.argmax(probabilities, axis = 0)

def plot_cost_function_over_time(cost_function_history):
    plt.plot(range(len(cost_function_history)), cost_function_history)
    plt.ylabel('Cost Function')
    plt.xlabel('Iteration number')
    plt.show()

def compute_test_error(X, Y, theta, temp_parameter):
    error_count = 0.
    assigned_labels = get_classification(X, theta, temp_parameter)
    return 1 - np.mean(assigned_labels == Y)

In [371]:
def compute_test_error_mod3(X, Y, theta, temp_parameter):
    """
    Returns the error of these new labels when the classifier predicts the digit. (mod 3)

    Args:
        X - (n, d - 1) NumPy array (n datapoints each with d - 1 features)
        Y - (n, ) NumPy array containing the labels (a number from 0-2) for each
            data point
        theta - (k, d) NumPy array, where row j represents the parameters of our
                model for label j
        temp_parameter - the temperature parameter of softmax function (scalar)

    Returns:
        test_error - the error rate of the classifier (scalar)
    """
    
    y_own = get_classification(X, theta, temp_parameter) % 3
    error = 0
    for x, b in zip(Y, y_own):
        error += (x != b)

    return (error / len(Y))


array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0])

In [386]:
y_own = get_classification(X[:, 0:-1], theta, temp_parameter)

In [403]:
def center_data(X):
    """
    Returns a centered version of the data, where each feature now has mean = 0

    Args:
        X - n x d NumPy array of n data points, each with d features

    Returns:
        - (n, d) NumPy array X' where for each i = 1, ..., n and j = 1, ..., d:
        X'[i][j] = X[i][j] - means[j]       
    - (d, ) NumPy array with the columns means

    """
    feature_means = X.mean(axis=0)
    return (X - feature_means), feature_means


def principal_components(centered_data):
    """
    Returns the principal component vectors of the data, sorted in decreasing order
    of eigenvalue magnitude. This function first calculates the covariance matrix
    and then finds its eigenvectors.

    Args:
        centered_data - n x d NumPy array of n data points, each with d features

    Returns:
        d x d NumPy array whose columns are the principal component directions sorted
        in descending order by the amount of variation each direction (these are
        equivalent to the d eigenvectors of the covariance matrix sorted in descending
        order of eigenvalues, so the first column corresponds to the eigenvector with
        the largest eigenvalue
    """
    scatter_matrix = np.dot(centered_data.transpose(), centered_data)
    eigen_values, eigen_vectors = np.linalg.eig(scatter_matrix)
    # Re-order eigenvectors by eigenvalue magnitude:
    idx = eigen_values.argsort()[::-1]
    eigen_values = eigen_values[idx]
    eigen_vectors = eigen_vectors[:, idx]
    return eigen_vectors

In [433]:
def project_onto_PC(X, pcs, n_components, feature_means):
    """
    Given principal component vectors pcs = principal_components(X)
    this function returns a new data array in which each sample in X
    has been projected onto the first n_components principcal components.
    """
    # TODO: first center data using the feature_means
    # TODO: Return the projection of the centered dataset
    #       on the first n_components principal components.
    #       This should be an array with dimensions: n x n_components.
    # Hint: these principal components = first n_components columns
    #       of the eigenvectors returned by principal_components().
    #       Note that each eigenvector is already be a unit-vector,
    #       so the projection may be done using matrix multiplication.
    
    x_cent = np.ndarray(X.shape)
    for i in range(0, len(feature_means)):
        x_cent[:, i] = X[:, i] - feature_means[i]

    proj = x_cent @ pcs[:, 0:n_components]

    return proj


In [424]:
X = np.array([
    [1, 2, 3],
    [2, 4, 6],
    [3, 6, 9],
    [4, 8, 12],
]);
x_centered, feature_means = center_data(X)
pcs = principal_components(x_centered)
exp_res = np.array([
    [5.61248608, 0],
    [1.87082869, 0],
    [-1.87082869, 0],
    [-5.61248608, 0],
])
n_components = 2

In [427]:
project_onto_PC(X, pcs, n_components, feature_means)

array([[-2.67261242e-01, -8.99989016e-01, -2.59226735e-16],
       [-5.34522484e-01, -1.58890294e-01, -8.32050294e-01],
       [-8.01783726e-01,  4.05923201e-01,  5.54700196e-01]])

In [434]:
def cubic_features(X):
    """
    Returns a new dataset with features given by the mapping
    which corresponds to the cubic kernel.
    """
    n, d = X.shape  # dataset size, input dimension
    X_withones = np.ones((n, d + 1))
    X_withones[:, :-1] = X
    new_d = 0  # dimension of output
    new_d = int((d + 1) * (d + 2) * (d + 3) / 6)

    new_data = np.zeros((n, new_d))
    col_index = 0
    for x_i in range(n):
        X_i = X[x_i]
        X_i = X_i.reshape(1, X_i.size)

        if d > 2:
            comb_2 = np.matmul(np.transpose(X_i), X_i)

            unique_2 = comb_2[np.triu_indices(d, 1)]
            unique_2 = unique_2.reshape(unique_2.size, 1)
            comb_3 = np.matmul(unique_2, X_i)
            keep_m = np.zeros(comb_3.shape)
            index = 0
            for i in range(d - 1):
                keep_m[index + np.arange(d - 1 - i), i] = 0

                tri_keep = np.triu_indices(d - 1 - i, 1)

                correct_0 = tri_keep[0] + index
                correct_1 = tri_keep[1] + i + 1

                keep_m[correct_0, correct_1] = 1
                index += d - 1 - i

            unique_3 = np.sqrt(6) * comb_3[np.nonzero(keep_m)]

            new_data[x_i, np.arange(unique_3.size)] = unique_3
            col_index = unique_3.size

    for i in range(n):
        newdata_colindex = col_index
        for j in range(d + 1):
            new_data[i, newdata_colindex] = X_withones[i, j]**3
            newdata_colindex += 1
            for k in range(j + 1, d + 1):
                new_data[i, newdata_colindex] = X_withones[i, j]**2 * X_withones[i, k] * (3**(0.5))
                newdata_colindex += 1

                new_data[i, newdata_colindex] = X_withones[i, j] * X_withones[i, k]**2 * (3**(0.5))
                newdata_colindex += 1

                if k < d:
                    new_data[i, newdata_colindex] = X_withones[i, j] * X_withones[i, k] * (6**(0.5))
                    newdata_colindex += 1

    return new_data

In [442]:
a = np.array([[1, 0], [0, 1]])

In [443]:
cubic_features(a)

array([[1.        , 0.        , 0.        , 0.        , 1.73205081,
        1.73205081, 0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 1.73205081, 1.73205081, 1.        ]])

In [446]:
def polynomial_kernel(X, Y, c, p):
    """
        Compute the polynomial kernel between two matrices X and Y::
            K(x, y) = (<x, y> + c)^p
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            c - a coefficient to trade off high-order and low-order terms (scalar)
            p - the degree of the polynomial kernel

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    raise NotImplementedError
